# Exploration of BRO REST API
- [Documentation - BRO productomgeving](https://www.bro-productomgeving.nl/bpo/latest/url-s-publieke-rest-services)
- [REST API documentation](https://publiek.broservices.nl/gm/gld/v1/swagger-ui/)

In [9]:
import requests
headers = {
    "accept": "text/plain"
}

In [10]:
GLD_ID = "GLD000000054694" # BRO-ID
response = requests.get(f"https://publiek.broservices.nl/gm/gld/v1/seriesAsCsv/{GLD_ID}", headers=headers)

The response is plain text of csv data of the time series.

In [3]:
response.text

'Tijdstip,Voorlopige Waarde [m],Voorlopige Opmerking,Beoordeelde Waarde [m],Beoordeelde Opmerking,Controle Waarde [m],Controle Opmerking,Onbekend Waarde [m],Onbekend Opmerking\n-543243600000,,,,,,,0.110,\n-537541200000,,,,,,,0.110,\n-534085200000,,,,,,,0.110,\n-532616400000,,,,,,,0.050,\n-531234000000,,,,,,,-0.220,\n-530197200000,,,,,,,-0.330,\n-528987600000,,,,,,,-0.750,\n-527518800000,,,,,,,-0.050,\n-526309200000,,,,,,,-0.500,\n-524926800000,,,,,,,-0.680,\n-523717200000,,,,,,,-0.770,\n-522162000000,,,,,,,-0.600,\n-520952400000,,,,,,,-0.780,\n-519656400000,,,,,,,-0.880,\n-518360400000,,,,,,,-0.600,\n-516978000000,,,,,,,-0.980,\n-515768400000,,,,,,,-0.470,\n-514299600000,,,,,,,-0.370,\n-513090000000,,,,,,,-0.110,\n-511707600000,,,,,,,-0.510,\n-510498000000,,,,,,,-0.630,\n-509029200000,,,,,,,-0.440,\n-507819600000,,,,,,,-0.370,\n-506437200000,,,,,,,-0.360,\n-505227600000,,,,,,,0.150,\n-503758800000,,,,,,,0.150,\n-500994000000,,,,,,,-0.560,\n-499784400000,,,,,,,0.040,\n-498574800000,,,,,

We can put the contents of the response in a pandas dataframe. Empty values are going to be `NaN`.
The columns and format of this file is different than the csv that one could download from the portal.

In [3]:
import pandas as pd
from io import StringIO

In [29]:
df = pd.read_csv(StringIO(response.text))

In [30]:
df

,Tijdstip,Voorlopige Waarde [m],Voorlopige Opmerking,Beoordeelde Waarde [m],Beoordeelde Opmerking,Controle Waarde [m],Controle Opmerking,Onbekend Waarde [m],Onbekend Opmerking
0,-543243600000,NaN,NaN,NaN,NaN,NaN,NaN,0.11,NaN
1,-537541200000,NaN,NaN,NaN,NaN,NaN,NaN,0.11,NaN
2,-534085200000,NaN,NaN,NaN,NaN,NaN,NaN,0.11,NaN
3,-532616400000,NaN,NaN,NaN,NaN,NaN,NaN,0.05,NaN
4,-531234000000,NaN,NaN,NaN,NaN,NaN,NaN,-0.22,NaN
...,...,...,...,...,...,...,...,...,...
261,-171291600000,NaN,NaN,NaN,NaN,NaN,NaN,-0.90,NaN
262,-169822800000,NaN,NaN,NaN,NaN,NaN,NaN,-1.06,NaN
263,-168613200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kleinerDanLimietwaarde
264,-167144400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kleinerDanLimietwaarde


In [ ]:
# remove columns whose name contains "Opmerking" (case-insensitive)
mask = df.columns.str.contains(r'Opmerking', case=False, na=False)
# remove columns containing NaNs
df_clean = df.loc[:, ~mask].dropna(axis=1, how='all')
# remove columns that contain Nans
df_clean.dropna(inplace=True)

In [32]:
df_clean

,Tijdstip,Onbekend Waarde [m]
0,-543243600000,0.11
1,-537541200000,0.11
2,-534085200000,0.11
3,-532616400000,0.05
4,-531234000000,-0.22
...,...,...
258,-175093200000,-0.60
259,-173797200000,-0.78
260,-172501200000,-0.82
261,-171291600000,-0.90


### Notes
By default the time given by bro with this query is in format: Unix epoch in milliseconds (can be negative): -519656400000 (pre‑1970 dates)
however, when fetching the data from the web app (BRO-loket) you can get a csv with the time in ISO 8601 with timezone: 1986-04-28T12:00:00+02:00 (tz-aware). To get ISO the query has to be modified as follows:
```python
- "https://publiek.broservices.nl/gm/gld/v1/seriesAsCsv/{GLD_ID}"
+ "https://publiek.broservices.nl/gm/gld/v1/seriesAsCsv/{GLD_ID}?asISO8601=asISO8601"
```

### Extension: querying metadata from well via BRO


In [11]:
response_object = requests.get(f"https://publiek.broservices.nl/gm/gld/v1/objectsAsCsv/{GLD_ID}?rapportagetype=volledig", headers=headers)

In [12]:
print(response_object.text)

"BRO-ID","bronhouder",,"kwaliteitsregime","datum eerste meting","datum recentste meting"
"GLD000000054694","50200097",,"IMBRO/A","(1952-10-14, JJJJ-MM-DD)","(1964-09-28, JJJJ-MM-DD)"
,,,,,
"put BRO-ID","put buisnummer",,"monitoringnet BRO-ID",,
"GMW000000052829","1",,,,
,,,,,
"observatie ID","start observatieperiode","eind observatieperiode","observatietype","mate beoordeling","observatieproces ID"
"OBS_DINO_RO_Alle","(1952-10-14, JJJJ-MM-DD)","(1964-09-28, JJJJ-MM-DD)","reguliereMeting","onbekend","OPR_RO_Alle"
,,,,,
"tijdstip meting","waterstand","status kwaliteitscontrole","censuurreden","censuurlimietwaarde","interpolatietype"
"1952-10-14T12:00:00+01:00","0.110","onbekend",,,"discontinu"
"1952-12-19T12:00:00+01:00","0.110","onbekend",,,"discontinu"
"1953-01-28T12:00:00+01:00","0.110","onbekend",,,"discontinu"
"1953-02-14T12:00:00+01:00","0.050","onbekend",,,"discontinu"
"1953-03-02T12:00:00+01:00","-0.220","onbekend",,,"discontinu"
"1953-03-14T12:00:00+01:00","-0.330","onbekend",,,

In [13]:
import re
m = re.search(r'\b(GMW[0-9]+)\b', response_object.text)
print(m)

<re.Match object; span=(246, 261), match='GMW000000052829'>


In [14]:
gmw_id = m.group(1)

In [15]:
# Get coordinates of a well (GMW)
import xml.etree.ElementTree as ET
response_gmw = requests.get(f"https://int-publiek.broservices.nl/gm/gmw/v1/objects/{gmw_id}?fullHistory=", headers= {"accept": "application/xml"})
xml = response_gmw.text
root = ET.fromstring(xml)
ns = {
    "gml": "http://www.opengis.net/gml/3.2",
    "gmwcommon": "http://www.broservices.nl/xsd/gmwcommon/1.1",
    "brocom": "http://www.broservices.nl/xsd/brocommon/3.0",
    "dsgmw": "http://www.broservices.nl/xsd/dsgmw/1.1",
}
pos = root.find(".//gmwcommon:location/gml:pos", ns)
x, y = map(float, pos.text.split())
print(x, y)

266760.0 524240.0
